![video decomposition clustering](https://raw.githubusercontent.com/ARVEST-APP/ml-notebooks/refs/heads/main/docs/images/notebooks/video-decomposiiton-clustering.png)

In this notebook, we shall start learning about the [Arvest](https://arvest.app/en) API and its [python package](https://github.com/ARVEST-APP/arvest-api) by taking some audiovisual IIIF Manifests sorted on Arvest, decomposing them into images, and using clustering and other machine learning techniques to create some interactive interfaces that we can upload and use in Arvest. 

# 0. Setup

Let's begin by installing and importing all of the different components we will need.

In [ ]:
print("Installing and importing packages...")

# Uninstall and reinstall packages for a clean environment
!pip uninstall -q -y arvestapi
!pip uninstall -q -y arvesttools
!pip uninstall -q -y jhutils
!pip uninstall -q -y iiif_prezi3
!pip uninstall -q -y dvt
!pip install -q --disable-pip-version-check git+https://github.com/ARVEST-APP/arvest-api.git
!pip install -q --disable-pip-version-check git+https://github.com/ARVEST-APP/arvest-api-tools.git
!pip install -q --disable-pip-version-check git+https://github.com/jdchart/jh-py-utils.git
!pip install -q --disable-pip-version-check git+https://github.com/iiif-prezi/iiif-prezi3.git
!pip install -q --disable-pip-version-check git+https://github.com/distant-viewing/dvt.git
!pip install -q --disable-pip-version-check opencv-python
!pip install -q --disable-pip-version-check scikit-learn
!pip install -q --disable-pip-version-check matplotlib

# Import packages
import arvestapi
import arvesttools.manifest_creation
from jhutils.local_files import read_json, write_json, get_image_info, collect_files
import jhutils.online_files
from jhutils.misc import print_progress_bar_colab, slugify
import os
import dvt
import iiif_prezi3
import shutil
import requests
import json
import cv2
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from PIL import Image
import random
import mimetypes

print("👍 Ready!")

Installing and importing packages...
👍 Ready!


### If you're following a workshop session...

If you're currently in a workshop - hello! 👋

You can run this cell to download all of the data into your colab session so that you don't have to run all of the time-consuming processes.

In [2]:
IM_IN_A_WORKSHOP = False

if IM_IN_A_WORKSHOP:
    print("Downloading data...")
    
    # Download and unpack zip:
    os.makedirs(os.path.join(os.getcwd(), "downloaded_data"), exist_ok=True)
    jhutils.online_files.download_zip("https://github.com/ARVEST-APP/arvest-workshops/raw/main/resources/workshop-dl-media.zip", os.path.join(os.getcwd(), "downloaded_data"))
    print("👍 Downloaded data")

## 1. Connect to Arvest

First, we need to "connect" to Arvest using the Arvest API package. For this, we need our user email and our password which we will give to an instance of the `arvestapi.Arvest()` class. For convenience, we've saved ours in a file which is why we get `LOGIN_DATA` by reading a json file.

In [ ]:
# First, let's connect to our Arvest account:
LOGIN_DATA = os.path.join(os.getcwd(), "login_private.json")
credentials = read_json(LOGIN_DATA)

ar = arvestapi.Arvest(credentials["email"], credentials["password"])
print(f"👍 Succesfully connected to Arvest with \"{ar.profile.name}\"")

# 2. Get videos
First we will need some sources to process! In this example, we shall be comparing different productions of Pina Bausch's [_Café Müller_](https://en.wikipedia.org/wiki/Caf%C3%A9_M%C3%BCller).

It so happens that we have some video Manifests on our Arvest account. Let's get our Manifests using the Arvest API's `get_manifests()` function.

Once e have all of our Manifests, we can cycle through them, and check if each one is one of the Manifests that we're looking for. We know that our Manifest items have a specific description, so lets check that by inspecting the `description` property.

In [ ]:
VIDEO_MANIFESTS = []

# Get all of our manifests:
manifests = ar.get_manifests()

# Get all of the manifests with a given description
for manifest in manifests:
    if manifest.description == "Café Müller video recording":
        VIDEO_MANIFESTS.append(manifest)

print(f"🔍 Found {len(VIDEO_MANIFESTS)} manifests.")

### Download the videos for processing

In order to process the videos we will need to be able to access them locally, which means that we will need to download them into our session. We can do this by getting the Manifest content, and then retrieving the `id` of the video of the first canvas.

ℹ️ This step will be skipped if you downloaded the workshop elements, however you cans till run the cell.

In [ ]:
LOCAL_VIDEO_PATHS = {}

if not IM_IN_A_WORKSHOP:
    for i, manifest in enumerate(VIDEO_MANIFESTS):
        video_url = manifest.get_content()["items"][0]["items"][0]["items"][0]["body"]["id"]
        print_progress_bar(i + 1, len(VIDEO_MANIFESTS), f"(downloading {video_url})...")

        path = jhutils.online_files.download(video_url, dir = os.path.join(os.getcwd(), "data", "videos"))
        LOCAL_VIDEO_PATHS[slugify(manifest.get_content()["id"])] = path
else:
    LOCAL_VIDEO_PATHS = {} # ADD THIS

print(f"👍 Downloaded videos")

## 3. Extract images

We're going to process one image for every second of video in order to get a good idea about the visual composition of each video. To do this we'll first need to extract images from the videos.

ℹ️ This step will be skipped if you downloaded the workshop elements, however you cans till run the cell.

In [ ]:
LOCAL_IMAGE_PATHS = {}
INTERVAL = 1
img_count = 0

if not IM_IN_A_WORKSHOP:
    for i, video_name in enumerate(LOCAL_VIDEO_PATHS):
        print_progress_bar_colab(i + 1, len(LOCAL_VIDEO_PATHS), f"(treating {video_name})...")

        # Output folder:
        out_folder = os.path.join(os.getcwd(), "data", "images", video_name)
        os.makedirs(out_folder, exist_ok=True)
        LOCAL_IMAGE_PATHS[video_name] = out_folder

        # Open video file with opencv 2 and get properties
        cap = cv2.VideoCapture(LOCAL_VIDEO_PATHS[video_name])
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        # For iteration:
        frame_interval = int(fps * INTERVAL)
        frame_num = 0
        saved_frame_count = 0

        while frame_num < total_frames:
            # Get frame
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()
            if not ret:
                break
            
            # Save image
            timestamp_secs = frame_num / fps
            output_path = os.path.join(out_folder, f"frame_{saved_frame_count:04d}_{timestamp_secs:.2f}.jpg")
            cv2.imwrite(output_path, frame)
            img_count = img_count + 1

            saved_frame_count += 1
            frame_num += frame_interval

        cap.release()
else:
    LOCAL_IMAGE_PATHS = {} # ADD THIS

print(f"🏞️ Finished extracting {img_count} images!")

# 4. Process embeddings
Next we can use the [distant viewing toolkit](https://github.com/distant-viewing/dvt) to map the images within an embedding space. 

We'll save the embedding data as a **numpy file** (`.npy`) so that we don't have to run this step again.

ℹ️ If you are running this for the first time dvt it will download the model onto your computer.
ℹ️ This step will be skipped if you downloaded the workshop elements, however you cans till run the cell.

In [ ]:
print("Processing...")

# Instance of dvt AnnoEmbed class:
embedder = dvt.AnnoEmbed()

if not IM_IN_A_WORKSHOP:
    first_time = True
    for i, video_name in enumerate(LOCAL_IMAGE_PATHS):

        corpus = collect_files(LOCAL_IMAGE_PATHS[video_name], ["jpg"])

        for i, image_file in enumerate(corpus):
            print_progress_bar_colab(i + 1, len(corpus), f"Treating {os.path.basename(image_file)}")

            image_as_np = cv2.imread(image_file)
            image_as_np = cv2.cvtColor(image_as_np, cv2.COLOR_BGR2RGB)

            embedding = embedder.run(image_as_np)["embedding"]
            if first_time:
                embedding_list = embedding
                first_time = False
            else:
                embedding_list = np.vstack((embedding_list, embedding))

    embedding_file = os.path.join(os.getcwd(), "data", "embeddings.npy")
    np.save(embedding_file, embedding_list)

print("👍Finished collecting embeddings!")

# 3. Dimensionality reduction and clustering
Now that we have our embedding data, we can use dimensionality reduction to crunch all of these dimensions down into 2 so that they can be projected into a 2-dimensional space. 

To do this, we'll use the dimensionality reduction algorithm [T-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding). 

Note that we also do some pre- and post-processing, the full process is: standardisation -> dimensionality reduction -> normalization.

In [3]:
if not IM_IN_A_WORKSHOP:
    embedding_file = os.path.join(os.getcwd(), "data", "embeddings.npy")
else:
    embedding_file = "" # ADD THIS
    
embedding_list = np.load(embedding_file)

standardized = StandardScaler().fit_transform(embedding_list)

tsne = TSNE(n_components = 2, perplexity = 5, learning_rate=10, n_iter=250)
reduced = tsne.fit_transform(standardized)

normalized = MinMaxScaler((0, 1)).fit_transform(reduced)

: 

If we like, we can visualize the data in a scatter plot:

In [ ]:
transposed = np.transpose(normalized)
plt.scatter(transposed[0], transposed[1])
plt.show()

## Clustering (optional)

Next we can perform some clustering on this data using [K-Means](https://en.wikipedia.org/wiki/K-means_clustering).

This will allow us to create subcategories.

In [ ]:
NUM_CLUSTERS = 6

kmeans = KMeans(n_clusters = NUM_CLUSTERS, random_state = 0, n_init = "auto")
clusters = kmeans.fit(normalized).labels_

# Create a random colour map for visualisation:
colour_map = {}
used = []
for item in clusters:
    if item not in used:
        colour_map[str(item)] = (random.random(), random.random(), random.random())
        used.append(item)

We can visualize the clusters in a scatter plot like this:

In [ ]:
transposed = np.transpose(normalized)
col = []
for item in clusters:
    col.append(colour_map[str(item)])

plt.scatter(transposed[0], transposed[1], c = col)
plt.show()

# 4. Export to Arvest
Finally, we shall export the results of our analysis to an image file, and create an annotated (and therefore interactive) IIIF Manifest that can be consulted in [Arvest](https://arvest.app/en). First, we shall create a high-res PNG file that projects the corresponding images into the 2D space of the dimensionality reduction. We shall also keep a track of the coordinates so that we can create our annotations later.

In [ ]:
IMAGE_PATH = os.path.join(os.getcwd(), "visualization-image.png")
COORDINATES_PATH = os.path.join(os.getcwd(), "visualization-coordinates.json")

WIDTH = 5000
HEIGHT = 5000
PADDING = 100
IMAGE_ZOOM = 0.1

def scale(val, old_min, old_max, new_min, new_max):
    return new_min + (((val - old_min) * (new_max - new_min)) / (old_max - old_min))

# Function for adding each image to the main image:
def add_image(full_image, coordinates_list, image_url, coordinates):
  img_path = os.path.join(TEMP_FOLDER, os.path.basename(image_url))
  img_data = get_image_info(img_path)
  this_img = Image.open(img_path)

  w = int(img_data["width"] * IMAGE_ZOOM)
  h = int(img_data["height"] * IMAGE_ZOOM)
  x = int(scale(int(int(float(coordinates[0]) * WIDTH) - (w * 0.5)), 0, WIDTH, PADDING, WIDTH - (PADDING * 2)))
  y = int(scale(int(int(float(coordinates[1]) * HEIGHT) - (h * 0.5)), 0, HEIGHT, PADDING, HEIGHT - (PADDING * 2)))

  this_img = this_img.resize((w, h))
  full_image.paste(this_img, (x, y))

  coordinates_list.append({"url" : image_url, "x" : x, "y" : y, "w" : w, "h" : h})

# Initialize image and coordinates
full_image = Image.new('RGBA', (WIDTH, HEIGHT))
coordinates = {"images" : []}

# Add all of the images:
for i, item in enumerate(normalized):
  image_data = corpus[i]
  print_progress_bar_colab(i, len(corpus) - 1, f"Treating {os.path.basename(image_data['url'])}")
  add_image(full_image, coordinates["images"], image_data['url'], item)

full_image.save(IMAGE_PATH)
write_json(COORDINATES_PATH, coordinates)

print("🎨 Image created!")

## Create Manifests
Now we need to create our Manifests. In order to make the main visualization Manifest truly interactive, we shall also make a _Manifest for each of the images in our corpus_. This must be done first, as we will need the URLs of these Manifests when creating our annotations.

First, we need to "connect" to Arvest using the Arvest API package. For this, we need our user email and our password which we will give to an instance of the `arvestapi.Arvest()` class. For convenience, we've saved ours in a file which is why we get `LOGIN_DATA` by reading a json file.

In [ ]:
# First, let's connect to our Arvest account:
LOGIN_DATA = os.path.join(os.getcwd(), "login_private.json")
credentials = read_json(LOGIN_DATA)

ar = arvestapi.Arvest(credentials["email"], credentials["password"])
print(f"👍 Succesfully connected to Arvest with \"{ar.profile.name}\"")

Next, we'll create our Manifests using the [arvesttools](https://github.com/ARVEST-APP/arvest-api-tools) package's helper function `media_to_manifest()`. We'll create a Manifest for each file in our corpus, and keep a track of the URLs which are created in the `MANIFEST_DICT` variable:

In [ ]:
MANIFEST_DICT = {}

for i, image_data in enumerate(corpus):
  print_progress_bar_colab(i + 1, len(corpus), f"Creating a Manifest for {os.path.basename(image_data['url'])}")

  img_path = os.path.join(TEMP_FOLDER, os.path.basename(image_data['url']))
  img_filename = os.path.splitext(os.path.basename(image_data['url']))[0]
  img_data = get_image_info(img_path)

  # Create the iiif_prezi3.Manifest:
  manifest = arvesttools.manifest_creation.media_to_manifest(img_path)

  # Update the ID to be the online location of the image:
  manifest.items[0].items[0].items[0].body.id = image_data['url']

  # Save the Manifest to disk
  local_path = os.path.join(TEMP_FOLDER, f"{slugify(img_filename)}.json")
  write_json(local_path, manifest.dict())

  # And upload to Arvest:
  added_manifest = ar.add_manifest(path = local_path, update_id = True)
  added_manifest.update_title(f"{img_filename}")
  added_manifest.update_description("Local view of an image embedding projection.")
  
  manifest_metadata = added_manifest.get_metadata()
  manifest_metadata["creator"] = "Image embedding projection tutorial"
  manifest_metadata["identifier"] = "&&API-TUTORIAL-IMAGE-EMBEDDING"
  added_manifest.update_metadata(manifest_metadata)

  # Keep track of the urls that are created:
  MANIFEST_DICT[image_data['url']] = added_manifest.get_full_url()

print("👍 Finished!")

Finally, let's create the main visualization Manifest. First, we need to upload the image we created of the projection to Arvest. For this, we'll use the `add_media()` function.

In [ ]:
added_media = ar.add_media(path = IMAGE_PATH)
added_media.update_title("Image collection projection")
added_media.update_description("A projection in 2D space of a collection of images.")

media_metadata = added_media.get_metadata()
media_metadata["creator"] = "Image embedding projection tutorial"
media_metadata["identifier"] = "&&API-TUTORIAL-IMAGE-EMBEDDING"
added_media.update_metadata(media_metadata)

print(f"👍 Media uploaded to Arvest at the following url: {added_media.get_full_url()}")

Now let's create the Manifest. Again, we'll use the `media_to_manifest()` function, which in this case can also accept an Arvest media item. 

Once we've created the Manifest, we can add annotations to render it interactive. We'll add an annotation for each of the Manifests  created earlier using the `add_textual_annotation()` function with the corresponding Manifest url and spatial position and dimensions:

In [ ]:
# Create the Manifest:
manifest = arvesttools.manifest_creation.media_to_manifest(added_media)

# Add an annotation for each Manifest:
for item in coordinates["images"]:
    image_url = item["url"]
    manifest_url = MANIFEST_DICT[image_url]
    xywh = {"x" : item["x"], "y" : item["y"], "w" : item["w"], "h" : item["h"]}
    
    arvesttools.manifest_creation.add_textual_annotation(
        manifest,
        text_content = f"<p>{os.path.basename(image_url)}</p>",
        xywh = xywh,
        linked_manifest = manifest_url
    )

# Save to disk:
local_path = os.path.join(TEMP_FOLDER, "projection-manifest.json")
write_json(local_path, manifest.dict())

# And upload to Arvest:
added_manifest = ar.add_manifest(path = local_path, update_id = True)
added_manifest.update_title("Image embedding projection")
added_manifest.update_description("Projection of a collection of images in 2-D space.")

manifest_metadata = added_manifest.get_metadata()
manifest_metadata["creator"] = "Image embedding projection tutorial"
manifest_metadata["identifier"] = "&&API-TUTORIAL-IMAGE-EMBEDDING"
added_manifest.update_metadata(manifest_metadata)

print(f"👍 Manifest uploaded to Arvest at the following url: {added_manifest.get_preview_url()}")

# 5. Cleanup
To finish, lets clean up our mess! First, we can delete the temporary folder .

In [ ]:
shutil.rmtree(TEMP_FOLDER)
os.remove(IMAGE_PATH)
os.remove(COORDINATES_PATH)
print(f"🗑️ {TEMP_FOLDER} removed !")

And finally, we can remove from Arvest all of our content. We can get all of our content by using the `get_manifests()` and `get_medias()` functions, then check the metadata. If it's one of the files we want to remove, we can then use the `remove()` function.

**⚠️ Warning: there's no going back after using the remove function, so be careful! To avoid accidential removal, we've added a `REMOVE` variable that need to be set to `True` for the code to run.**

In [ ]:
REMOVE = True

if REMOVE:
    all_manifests = ar.get_manifests()
    count = 0
    print("Removing manifests...")

    for i, media_file in enumerate(all_manifests):
        print_progress_bar_colab(i + 1, len(all_manifests), f"(Processing file {i + 1}/{len(all_manifests)})")
        media_metadata = media_file.get_metadata()
        if media_metadata["creator"] == "Image embedding projection tutorial" and media_metadata["identifier"] == "&&API-TUTORIAL-IMAGE-EMBEDDING":
            media_file.remove()
            count = count + 1

    all_media = ar.get_medias()
    print("Removing medias...")

    for i, media_file in enumerate(all_media):
        print_progress_bar_colab(i + 1, len(all_media), f"(Processing file {i + 1}/{len(all_media)})")
        media_metadata = media_file.get_metadata()
        if media_metadata["creator"] == "Image embedding projection tutorial" and media_metadata["identifier"] == "&&API-TUTORIAL-IMAGE-EMBEDDING":
            media_file.remove()
            count = count + 1

    print(f"🗑️ Removed {count} items!")